<center> <h1> <span style="color:black"> IA|BE Data Science Certificate - Module 3 - Computer lab 2  </h1> </center>



<center> <h2> <span style="color:red"> Building ML Model Applications with Flask</h1> </center>

# Introduction

Making inferences from your ML model via some sort of application is an important part of the process when you want users to be able to interact with your model in an intuitive way. This workshop will show you the basics on how to set up an inference app for your ML models via Flask.

[Flask](https://palletsprojects.com/p/flask/) is a web application framework, designed to get up and running quick and easy. This is a popular framework within Python and the [documentation](https://flask.palletsprojects.com/en/2.1.x/) shows how to get started fast. We will cover the basics of setting up a Flask application and how to route requests to the server.

We will be making use of [ngrok](https://ngrok.com), a very convenient tool to put your app on the internet. This tool allows to make your Flask apps running on localhost available on the internet via [flask-ngrok](https://github.com/gstaff/flask-ngrok). To follow along this lab you will need to create a free account on the [signup](https://dashboard.ngrok.com/signup) page, this only takes a minute.

# Technical setup

We first install the flask, flask-ngrok and pyngrok libraries via pip. The -q option quiets the output to keep the output clear.

In [ ]:
!pip install -q flask-ngrok
!pip install -q flask
!pip install -q pyngrok==4.1.1
#Needed flask==0.12.2 in the past as newer versions of flask didn't work in Colab, but issue seems fixed (see https://github.com/plotly/dash/issues/257)

You need to supply your personal ngrok authentication token to be able to run the app later on. You can find it in the "Your Authtoken" page when logged into ngrok online ([here](https://dashboard.ngrok.com/get-started/your-authtoken)).

Run the command `!ngrok authtoken insert_your_token_here_without_quotation_marks`

Fictional example: `!ngrok authtoken 28KM76XzAiQ3C0ASoq9mxDgKUam_6U7bKseK64vS9H54AMvrFalse`

In [ ]:
# Run the authtoken command here


Next, we import the libraries that we will use to build and serve our ML model.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

# Read data

We will create an ML model for the Ames housing data, se we first need to read our CSV data as a Pandas DataFrame.

In [ ]:
pd_ames = pd.read_csv("https://katrienantonio.github.io/hands-on-machine-learning-R-module-1/data/ames_python.csv")
pd_ames = pd_ames.drop(columns=["Unnamed: 0"])

After succesfully loading our data, we can inspect the first 10 rows with the Pandas `head` method.

In [ ]:
pd_ames.head(10)

# Data preperation

We perform some data preperation steps before fitting our ML model to the Ames housing data. The goal is to predict the sale price of a house, which has the following ditribution in our data:

In [ ]:
pd_ames.Sale_Price.hist(bins=20);

To make the target distribution more normal-like, we take the log transform:

In [ ]:
pd_ames["Log_Sale_Price"] = np.log(pd_ames["Sale_Price"])
pd_ames.Log_Sale_Price.hist(bins=20);

We learned in the previous part of this workshop that the log of the sale price (`Sale_Price`) is quite linearly related to the log of the lot area (`Lot_Area`), so let's add this transformed variable in our data:

In [ ]:
pd_ames["Log_Lot_Area"] = np.log(pd_ames["Lot_Area"])

# Build an ML model

We will create a very simple linear ML model which predicts the logged sale price (`Log_Sale_Price`) based on the logged lot area (`Log_Lot_Area`) and the living area (`Gr_Liv_Area`). The plots below show that these relations sort of hold:

In [ ]:
# Regression plot for logged lot area
sns.regplot(data=pd_ames, x="Log_Lot_Area", y="Log_Sale_Price");

In [ ]:
# Regression plot for living area
sns.regplot(data=pd_ames, x="Gr_Liv_Area", y="Log_Sale_Price");

Let's create our simple linear model via scikit-learn:

In [ ]:
# Create the feature matrix X and target vector y
X_ames = pd_ames[["Log_Lot_Area","Gr_Liv_Area"]]
y_ames = pd_ames.Log_Sale_Price
# Initialize the linear model and fit to our data
lin_mod = linear_model.LinearRegression()
lin_mod.fit(X_ames.values, y_ames.values)

The intercept of our model is equal to:

In [ ]:
lin_mod.intercept_

The coefficients of our model are equal to:

In [ ]:
pd.DataFrame({"feature":X_ames.columns.values,"coefficient":lin_mod.coef_})

We can use the `predict` method of our linear model to make inference on new houses:

In [ ]:
lin_mod.predict(np.array([[9,2000]]))

We can also define a function to make these calculations based on the intercept and coefficients:

In [ ]:
def ames_prediction(log_lot_area, gr_liv_area):
  return(lin_mod.intercept_ + log_lot_area*lin_mod.coef_[0] + gr_liv_area*lin_mod.coef_[1])

ames_prediction(9, 2000)

Inference is still done within our notebook. We now demonstrate how to set up a web application to make inference on our ML model via Flask.

# Create a basic Flask app

We will create a very basic "hello-world" type of Flask app to start. The following steps are taken:

* initiate the app by creating an instance of the `flask.Flask` class
* specify that you want to run the app with ngrok via the `run_with_ngrok` function
* define the functionality of our `welcome` function, which simply prints a welcome message
* run the app via the `.run()` method

Now you can click the hyperlink that ends on `ngrok.io` below and you will be redirected to your web application, where you should be able to see the welcome message. Cool, right?!

We skipped over one element in the code below so far, namely the `@route("/")` decorator for our welcome function. This route decorator tells Flask which URL should trigger our function. Setting this to "/", this will simply redirect us to the landing page of our application.

In [ ]:
# Initialize the app
app = Flask(__name__)
# Start ngrok when app is run
run_with_ngrok(app)

# Define our first app function
@app.route("/")
def welcome():
    return "Welcome to the wonderful world of ML inference via Flask!"

#if __name__ == '__main__':
app.run()  
# If address is in use, may need to terminate other sessions: Runtime > Manage Sessions > Terminate Other Sessions

## First basic inference

Let's now add another URL page to our web application. Below we add another function `inference` to our app, which for now simply prints another message for us. Note that the decorator for this function is chosen as `@app.route("/inference")`, so can navigate to this page by adding `/inference` to our application URL.

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def welcome():
    return "Welcome to the wonderful world of ML inference via Flask!"

# New function and URL added to our app
@app.route("/inference")
def inference():
    return "Let's do some inference on our Ames ML model..."

app.run()  

Our previous inference page did not really do any inference however. We now added a third function `inference_values` which takes two input arguments. These represent the two input features of our linear model, namely `Log_Lot_Area` and `Gr_Liv_Area`. Notice how the decorator for this fucntion is set to `@app.route("/inference/<int:a>,<int:b>")`. Try adding the following to the URL `/inference/9,2000`. What do you see? Does this number ring a bell?

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def welcome():
    return "Welcome to the wonderful world of ML inference via Flask!"

@app.route("/inference")
def inference():
    return "Let's do some inference on our Ames ML model..."

# An actual inference function is added to the app
@app.route("/inference/<int:a>,<int:b>")
def inference_values(a,b):
    pred_value = lin_mod.predict(np.array([[a,b]]))[0]
    return f"The predicted value is equal to {pred_value}"

app.run()

## Using template forms

Our previous attempt to perform inference was succesful, but not very flashy or user-friendly. Let's make the application nicer by using HTML templates for Flask forms. A Flask form is an HTML file with the `<form>` attribute:

```
<form action="action_to_perform_after_submission" method = "POST">
    <p>Field1 <input type = "text" name = "Field1_name" /></p>
    <p>Field2 <input type = "text" name = "Field2_name" /></p>
    ...
    <p><input type = "submit" value = "submit" /></p>
</form>
```

This HTML template decides the lay-out of the page to show. The action to perform is specified in the `action` attribute, which is what we will use to make our inference predictions.


We provide two template forms (`input_form.html` and `predict.html`) which are stored together in a `templates` folder. To be able to use these templates, you'll need to follow these steps:

* add the templates folder (containing both templates) in your Google Drive, aka My Drive
* mount your Google Drive in this notebook
* check that you can find the templates

The first step should be done manually, the last two steps are done with the following code:

In [ ]:
# Mount your Google Drive (you'll need to permit access when doing this)
from google.colab import drive
drive.mount('/content/gdrive')
# Check that you can find the templates
!ls /content/gdrive/MyDrive/templates

Let's first see how our template `input_form.html` renders in our app. We can use the `flask.render_template()` function with the template as input to generate the HTML page. Notice that our templates are stored in the "gdrive/MyDrive/templates" folder and we let Flask know via the `template_folder` argument.

In [ ]:
app = Flask(__name__, template_folder='gdrive/MyDrive/templates')
run_with_ngrok(app)

# Render the input form template at the homepage
@app.route("/")
def input_data():
  return render_template("input_form.html")
  
app.run()

We are not able to do inference yet after clicking the submit button, so let's add this functionality into our app now. We add the `predict_result` function which takes the input values that we supply and makes a prediction for our linear Ames model. The output is then rendered via the `predict.html` template.

The decorator is `@app.route("/predict",methods = ["POST"])` which shows that the result will be available at the `/predict` URL. Notice the `"POST"` method specification. We have two main HTTP methods to interact with the server:

* `GET`: pull specific info from the server, for example when a user request a page
* `POST`: send data from the user to the server, for example when a user fill in input data

Our `predict_result` function only allows a `POST` method, which will be invoked by us sending the house info via the submit button of out input form. Curious to see what happens when you try to access the `/predict` page directly?

In [ ]:
app = Flask(__name__, template_folder='gdrive/MyDrive/templates')
run_with_ngrok(app)

@app.route("/")
def input_data():
  return render_template("input_form.html")

# Implement a predict POST method to do some actual inference
@app.route("/predict",methods = ["POST"])
def predict_result():
  if request.method == "POST":
    to_predict_list = list(request.form.to_dict().values())
    prediction = str(lin_mod.predict(np.array([to_predict_list]))[0])
    return render_template("predict.html",prediction=prediction)
  
app.run()

Such annoying error messages might be scaring away users, so it is possible to also allow a `GET` method on our `/predict` page, which we catch by simply printing out a useful error message and fix to solve it.

In [ ]:
app = Flask(__name__, template_folder='gdrive/MyDrive/templates')
run_with_ngrok(app)

@app.route("/")
def input_data():
  return render_template("input_form.html")

# Add a predict GET method to propose a redirect instead of giving an error
@app.route("/predict",methods = ["GET","POST"])
def predict_result():
  if request.method == "GET":
    return "It seems like you are trying to reach the predict page without input values: go back to the '/' page."
  if request.method == "POST":
    to_predict_list = list(request.form.to_dict().values())
    prediction = str(lin_mod.predict(np.array([to_predict_list]))[0])
    return render_template("predict.html",prediction=prediction)
  
app.run()

Now you hopefully have a hang of the basics of web application development via Flask. Feel free to experiment further and add extra functionality or flashy items to your app. Have fun!

In [ ]:
# Your custom Flask app